# Get Amazon Data with Sampling

In [0]:
import pandas as pd

In [0]:
links = [
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Clothing_Shoes_and_Jewelry_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Clothing_Shoes_and_Jewelry_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Health_and_Personal_Care_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Tools_and_Home_Improvement_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Automotive_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Grocery_and_Gourmet_Food_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Patio_Lawn_and_Garden_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Baby_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz",
    "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz"
 
]

In [0]:
import pandas as pd

In [23]:
%%time
import random
chunkSize = 1000
SampleSize = 1000
StartIndez = random.randint(0,999)
EndIndex = StartIndez+1
print(StartIndez,EndIndex)
ReviewDf = pd.DataFrame()
for eachLink in links:
  print(eachLink)
  data = pd.read_json(eachLink, lines=True, chunksize = 100)
  for eachChunkindex, eachChunkValue in enumerate(data):
    if eachChunkindex in range(StartIndez,EndIndex):
      if len(eachChunkValue) <10000:
        SampleSize = 100
        ReviewDf = pd.concat([ReviewDf,eachChunkValue[['reviewText','overall','helpful']].sample(SampleSize)])
#   ReviewDf.append(data, ignore_index=True)
      else:
        ReviewDf = pd.concat([ReviewDf,eachChunkValue[['reviewText','overall','helpful']].sample(SampleSize)])
#   ReviewDf.append(data, ignore_index=True)
  
  
  

37 38
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Baby_5.json.gz
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_5.json.gz
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz
CPU times: user 21 s, sys: 272 ms, total: 21.2 s
Wall time: 34.9 s


In [24]:
len(ReviewDf)

400

In [25]:
ReviewDf.head()

,reviewText,overall,helpful
3784,"I like these bottles, they are good for breast...",5,"[0, 0]"
3754,These are very good bottles. I used them with ...,4,"[0, 0]"
3776,Baby is not nearly as gassy since we switched ...,5,"[1, 1]"
3783,these bottles were recommended by our pediatri...,5,"[0, 0]"
3766,I don't particularly understand the technology...,4,"[0, 0]"


In [0]:
ReviewDf.to_csv("AmazonReviewDump_23000.csv")

# Get individual Sentences 

In [27]:
data = pd.read_csv("AmazonReviewDump_23000.csv")
data.head(5)

,Unnamed: 0,reviewText,overall,helpful
0,0,Spiritually and mentally inspiring! A book tha...,5,"[0, 0]"
1,1,This is one my must have books. It is a master...,5,"[0, 2]"
2,2,This book provides a reflection that you can a...,5,"[0, 0]"
3,3,I first read THE PROPHET in college back in th...,5,"[0, 0]"
4,4,A timeless classic. It is a very demanding an...,5,"[7, 9]"


In [33]:
type(data['reviewText'][0])

str

In [0]:
data = data[data['reviewText'].notnull()]

In [0]:
def getIndSentence(eachValue):
  if eachValue:
    return eachValue.split(".")
  else:
    print (eachValue)
    
data['IndividualSentence'] = data['reviewText'].apply(getIndSentence)

In [42]:
data['IndividualSentence'][6]

["As you read, Gibran's poetry brings spiritual and visual beauty to life within you",
 ' Gibran is justly famous for rich metaphors that brilliantly highlight the pursuit of Truth and Goodness amidst all the darkness and light of human nature',
 '']

In [0]:
import itertools
FinalSentenceList = list(itertools.chain.from_iterable(data['IndividualSentence']))

In [45]:
len(FinalSentenceList)

185058

In [0]:
SentimentDf = pd.DataFrame({'sentence':FinalSentenceList})

In [54]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 1.9MB/s 


In [0]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


def getTextBlobSentiment(eachValue):
  return TextBlob(eachValue).sentiment

def getVaderSentiment(eachValue):
  return analyzer.polarity_scores(eachValue)

In [58]:
%time SentimentDf['textblob_sentiment'] = SentimentDf['sentence'].apply(getTextBlobSentiment)
%time SentimentDf['Vader_Sentiment'] = SentimentDf['sentence'].apply(getVaderSentiment)

CPU times: user 39.2 s, sys: 147 ms, total: 39.3 s
Wall time: 39.4 s


In [65]:
 len(SentimentDf[SentimentDf['sentence'].notnull()])

185058

In [66]:
SentimentDf.tail(1000)

,sentence,sentiment,Vader_Sentiment
184058,,"(0.0, 0.0)","{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
184059,"You need to give it a chance, like about 5 epi...","(0.0, 0.0)","{'neg': 0.0, 'neu': 0.609, 'pos': 0.391, 'comp..."
184060,The acting is good but the theme music grates...,"(0.35, 0.30000000000000004)","{'neg': 0.118, 'neu': 0.738, 'pos': 0.144, 'co..."
184061,It is harsh and has way too many horns betwee...,"(0.1952380952380952, 0.5785714285714286)","{'neg': 0.055, 'neu': 0.794, 'pos': 0.15, 'com..."
184062,I don't know how many seasons this show was o...,"(0.5, 0.55)","{'neg': 0.0, 'neu': 0.723, 'pos': 0.277, 'comp..."
184063,,"(0.0, 0.0)","{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
184064,Got my attention from the first episode to the...,"(0.125, 0.19999999999999998)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
184065,Very time appropriate for our current time i...,"(0.2333333333333333, 0.4000000000000001)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
184066,Just the right amount of drama and action fo...,"(0.19285714285714284, 0.31785714285714284)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
184067,Wish it could go on and on but know all good...,"(0.7, 0.6000000000000001)","{'neg': 0.0, 'neu': 0.725, 'pos': 0.275, 'comp..."
